# Notebook A: Toy fragment with SKQD (MVP)

In [ ]:
%pip -q install qiskit==2.1.* qiskit-aer qiskit-nature qiskit-algorithms

In [ ]:
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.transformers import FreezeCoreTransformer
from qiskit_aer.primitives import Estimator as AerEstimator
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from qiskit_nature.second_q.mappers import ParityMapper, QubitConverter

# Minimal H2 example as stand-in for toy fragment
driver = PySCFDriver(atom='H 0 0 0; H 0 0 0.74', basis='sto-3g', charge=0, spin=0, unit=DistanceUnit.ANGSTROM)
problem = driver.run()
problem = FreezeCoreTransformer().transform(problem)
second_q = problem.hamiltonian.second_q_op()

conv = QubitConverter(ParityMapper(), two_qubit_reduction=False)
Hq = conv.convert(second_q, num_particles=problem.num_particles)

psi0 = QuantumCircuit(Hq.num_qubits)
estimator = AerEstimator(backend_options={"method": "statevector", "device": "GPU"})
res = estimator.run([(psi0, [Hq], [{}])]).result()
E0 = res.values[0]
E0

In de echte SKQD-pipeline vervang je deze directe verwachting door een Krylov-basis en GEVD op schattingen van $H$ en $S$.